In [ ]:
] add Distributions

In [ ]:
using CSV
using DataFrames
using DataFramesMeta
using ODBC
using Distributions
using StatsBase

In [ ]:
bs = "bulk_bookstore_dataset.csv"

In [ ]:
df = CSV.read(read(bs), DataFrame);

In [ ]:
df = @chain df begin
    @rtransform $[:crap, :genre, :name] = begin
        x = split(:Breadcrumbs, "|")
        get.(Ref(x), 1:3, missing)
    end
end

In [ ]:
df.rating = rand(Uniform(1,5),nrow(df));

In [ ]:
aDf = CSV.read(read("iAuthors.csv"), DataFrame; header=["id", "FirstName", "LastName", "Biography", "PublisherId"]);

In [ ]:
aDf.author = [row.FirstName * ' ' * row.LastName for row in eachrow(aDf)];

In [ ]:
nrow(aDf)

In [ ]:
aDf = unique(aDf, :author);

In [ ]:
nrow(aDf)

In [ ]:
first(aDf,1)

In [ ]:
df.ISBN = [SubString(s,3,15) for s in df.ISBN];
df.ISBN = [parse(Int64,s) for s in df.ISBN];

In [ ]:
iDf = CSV.read(read("iBooks.csv"), DataFrame; header=[:id, :ISBN, :name, :Price, :rating, :GenreId, :PublisherId, :year,:copies, :description]);

In [ ]:
nrow(iDf)

In [ ]:
iDf = unique(iDf, :ISBN);

In [ ]:
nrow(iDf)

In [ ]:
first(iDf,1)

In [ ]:
authors = String[]
for row in eachrow(iDf)
    author = String(strip(df[only(findall(==(row.ISBN), df.ISBN)), :].Author))
    # println(author)
    author = split(author, ",")
    push!(authors, String(strip(author[1])))
end
# println(length(authors))
# println(nrow(iDf))    


In [ ]:
df.copies = rand(1000:1000000,nrow(df));

In [ ]:
# df.pub = [x.match for x in match.(r"^(\w+)", df.Publisher)]
df.PublisherId = [StatsBase.sample([1,2,3]) for _ in range(start=1, stop=nrow(df))];

In [ ]:
df.year = [parse(Int64,x.match) for x in match.(r"(\d{4})", df.Publisher)];

In [ ]:
odf = CSV.read(read("out.csv"), DataFrame);

In [ ]:
df.description = odf[:,"description"];

In [ ]:
# df.ISBN = [SubString(s,3,15) for s in df.ISBN];

In [ ]:
# df.ISBN = [parse(Int64,s) for s in df.ISBN];

In [ ]:
df = df[completecases(df),:]
disallowmissing!(df);

In [ ]:
nSet = Set()

In [ ]:
pdf = DataFrame()
for row in eachrow(df)
    twoNames = split(row.Author, ",")
    for name in twoNames
        name = strip(String(name))
        row.Author = name 
        push!(pdf, row)
        push!(nSet, name)
    end
end        

In [ ]:
saDf = DataFrame([:AuthorId=>[], :Author=>[]])
for (i, val) in enumerate(nSet)
    push!(saDf, [i strip(val)])
    # println(nrow(filter(:Author => n -> n == val, df)))
    # filter(:Author => n -> n == val, df).AuthorId .= i
    # df[:AuthorId, filter(:Author => n -> n == val, df)] .= i
    # @. df.AuthorId = ifelse(df.Author == val, i, df.AuthorId)
    # println(filter(:Author => n -> n == val, df).AuthorId)
    # println(i)
end

In [ ]:
print(nrow(saDf))

In [ ]:
afiles = readdir("./au", join=true)

In [ ]:
adf = CSV.read(read(afiles[1]), DataFrame)
for file in afiles[2:end]
    adf =  vcat(adf, CSV.read(read(file), DataFrame));
end    

In [ ]:
adf = adf[completecases(adf),:];

In [ ]:
nrow(saDf)

In [ ]:
function parseFirstName(s::String)::String
    names = split(s, " ")
    if length(names) == 3
        return names[1] * " " * names[2]
    end
    return names[1]
end

In [ ]:
function parseLastName(s::String)::String
    names = split(s, " ")
    if length(names) == 3
        return names[3]
    end
    if length(names) < 2
        println(s)
        return ""
    end
    return names[2]
end

In [ ]:
ISBN = 9780060001261
pdf[findall(==(ISBN), pdf.ISBN), :].Author

In [ ]:
aDf = DataFrame([:id=>[], :FirstName=>[], :LastName=>[], :Biography=>[], :PublisherId=>[], :Author=>[]])
for row in eachrow(adf)
    authors =  pdf[findall(==(row.ISBN), pdf.ISBN), :].Author
    for author in eachrow(authors)
        author = String(strip(String(author[1])))
        try
            AuthorId = saDf[only(findall(==(author), saDf.Author)), :].AuthorId
            push!(aDf, [AuthorId parseFirstName(author) parseLastName(author) row.biography 1 author])
        catch e
            println(e)
            println(author)
            continue
        end    
    end
end    

In [ ]:
LastName="Murphy"
aDf[findall(==(LastName), aDf.LastName), :]

In [ ]:
df = filter(:Author => i -> (i ∈ aDf.Author), pdf);

In [ ]:
nrow(df)
# df.AuthorId =[aDf[only(findall(==(row.Author), aDf.author)), :].id for row in eachrow(df)];

In [ ]:
genres = CSV.read(read("genres.csv"), DataFrame);

In [ ]:
df.GenreId = [rownumber(genres[only(findall(==(row.genre), genres.genre)), :]) for row in eachrow(df)];

In [ ]:
df.Price = [parse(Float64, SubString(s,2)) for s in df.Price];

In [ ]:
nrow(df)

In [ ]:
df = unique(df, :ISBN);

In [ ]:
nrow(df)

In [ ]:
CSV.write(".\\books.csv", df[:,[:ISBN, :name, :Price, :rating, :GenreId, :PublisherId, :year,:copies, :description]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
nrow(aDf)

In [ ]:
aDf = unique(aDf, :id);

In [ ]:
nrow(aDf)

In [ ]:
CSV.write(".\\authors.csv", aDf[:,[:id, :FirstName, :LastName, :Biography, :PublisherId]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
df = CSV.read(read("iBooks.csv"), DataFrame; header=[:id, :ISBN, :name, :Price, :rating, :GenreId, :PublisherId, :year,:copies, :description]);

In [ ]:
book_authors = DataFrame([:BookId=>[],:AuthorId=>[]] )

In [ ]:
author = "Stuart J. Murphy"
authors[findall(==(author), authors.author), :]

In [ ]:
first(aDf,1)

In [ ]:
first(pdf,1)

In [ ]:
for row in eachrow(df)
    try
        isbns = pdf[findall(==(row.ISBN), pdf.ISBN), :]
        for isbn in eachrow(isbns)
            author = strip(string(isbn.Author))
            aids = authors[findall(==(author), authors.author), :].id
            for aid in eachrow(aids)
                push!(book_authors,[row.id aid]);  
            end
        end
    catch e
        println(e)
        # println(row)
        println("shit")
        continue
    end   
    # fn = aDf[only(findall(==(row.AuthorId), aDf.id)), :].FirstName 
    # ln = aDf[only(findall(==(row.AuthorId), aDf.id)), :].LastName 
    # a = fn * ' ' * ln
    # push!(nSet,a)
end

In [ ]:
book_authors = unique(book_authors);

In [197]:
nrow(book_authors)

381

In [ ]:
nrow(aDf)

In [198]:
CSV.write(".\\book_authors.csv", book_authors)

".\\book_authors.csv"

In [ ]:
df = CSV.read(read("iBooks.csv"), DataFrame; header=[:id, :ISBN, :name, :Price, :rating, :GenreId, :PublisherId, :year,:copies, :description]);

In [ ]:
df.PublisherId = [StatsBase.sample([1,2,3]) for _ in range(start=1, stop=nrow(df))];

In [ ]:
CSV.write(".\\books.csv", df[:,[:ISBN, :name, :Price, :rating, :GenreId, :PublisherId, :year,:copies, :description]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
ba = CSV.read(read("book_authors.csv"), DataFrame);

In [ ]:
first(ba,5)

In [ ]:
aDf = CSV.read(read("iAuthors.csv"), DataFrame; header=["id", "FirstName", "LastName", "Biography", "PublisherId"]);

In [ ]:
#df = CSV.read(read("iBooks.csv"), DataFrame; header=[:id, :ISBN, :name, :Price, :rating, :GenreId, :PublisherId, :year,:copies, :description]);
nrow(aDf)

In [ ]:
aDf = unique(aDf, :id);

In [ ]:
nrow(aDf)

In [ ]:
first(aDf,5)

In [ ]:
first(df,5)

In [ ]:
pubI = Int64[]

In [ ]:
for row in eachrow(ba)
    # print(row.AuthorId)
    author = aDf[only(findall(==(row.AuthorId), aDf.id)), :]
    book = df[only(findall(==(row.BookId), df.id)), :]
    # push!(pubI, book.PublisherId)
    aDf[only(findall(==(row.AuthorId), aDf.id)), :].PublisherId = book.PublisherId
end     

In [ ]:
CSV.write(".\\authors.csv", aDf[:,[:id, :FirstName, :LastName, :Biography, :PublisherId]]; openquotechar='"', closequotechar='"', escapechar='"', quotestrings=true)

In [ ]:
df.PublisherId = [StatsBase.sample([1,2,3]) for _ in range(start=1, stop=nrow(df))];

In [ ]:
bbook_authors =  CSV.read(read("book_authors.bigger.csv"), DataFrame);

In [ ]:
cbook_authors = vcat(bbook_authors, book_authors);

In [ ]:
book_authors = unique(cbook_authors);

In [ ]:
nrow(book_authors)

In [ ]:
oauthors = CSV.read(read("authors.old.csv"), DataFrame);

In [ ]:
authors = CSV.read(read("authors.csv"), DataFrame);

In [ ]:
authors = vcat(authors,oauthors);

In [ ]:
first(authors,5)

In [ ]:
authors = unique(authors, :id);

In [ ]:
nrow(authors)

In [ ]:
authors = unique(authors, [:FirstName,:LastName]);

In [ ]:
CSV.write(".\\authors.csv",authors)

In [ ]:
authors[findall(==("Mark"), authors.FirstName), :]

In [ ]:
authors.author = [a.FirstName * " " * a.LastName for a in eachrow(authors)]; 

In [ ]:
first(authors,5)

In [ ]:
authors = authors[completecases(authors),:]
disallowmissing!(authors);